### **Practise I**

**1. Sending a Command to a Mining Drone**

Create a simple form (using WTF-Forms) to send a text command to an autonomous drone.

1.  **Create the form** in `forms.py` named `DroneCommandForm`. It will contain:
    * `drone_id` (a `StringField`, required, for the drone's identification, e.g., "DR-7")
    * `command` (a `StringField`, required, for the command itself, e.g., "SCAN_SECTOR_GAMMA")
    * `submit` (a submit button)

2.  **Create a route** `/send_command` that will display an HTML page (e.g., `drone_command.html`) with our form and will handle both methods (`GET` and `POST`).

3.  After a successful submission (`validate_on_submit`), the form will return a simple confirmation message, for example:
    `f"Command '{form.command.data}' has been sent to drone '{form.drone_id.data}'."`

In [ ]:
# configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' # setting the "secret key"

In [ ]:
from config.configuration import Config
from flask import Flask, render_template
from forms import DroneCommandForm # import our form class


app = Flask(__name__)

app.config.from_object(Config)


@app.route("/send_command", methods=["GET", "POST"])
def send_command():
    form = DroneCommandForm()
    if form.validate_on_submit():
        return f"Command '{form.command.data}' has been sent to drone '{form.id.data}'."
    return render_template("drone_command.html", form=form)


if __name__ == "__main__":
    app.run()

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class DroneCommandForm(FlaskForm):
    id = StringField(label="Dron ID", validators=[validators.DataRequired()])
    command = StringField(label="Command", validators=[validators.DataRequired()])
    submit = SubmitField(label="Send command")

In [ ]:
<!-- drone_command.html -->

<form method = "POST" action = "{{ url_for('send_command') }}">
    <p>Drone command form</p>
    {{ form.hidden_tag() }} <!-- CSRF token -->
    <p>{{ form.id.label }} {{ form.id(size=30) }}</p>
    <p>{{ form.command.label }} {{ form.command(size=30) }}</p>
    <p>{{ form.submit() }}</p>
</form>

### **Practise II**

**2. Energy Shield Calibration**

Create an interface to change the power level of the station's energy shield.

**Data:**
At the beginning of your file, create a simple dictionary that represents the current shield settings.
```python
SHIELD_SETTINGS = {'power_level': 50, 'frequency': 75.5}
```

1.  **Create a form** `ShieldForm` with one field:
    * `new_power_level` (an `IntegerField`, required)
    * `submit` (a submit button)

2.  **Create a route** `/shield_calibration` (`GET` and `POST`).
    * On a `GET` request, it will display a template (e.g., `shield_calibration.html`) that shows the **current** `power_level` from the `SHIELD_SETTINGS` dictionary and also displays the form for changing it.
    * On a `POST` request (`validate_on_submit`):
        * It updates the `'power_level'` value in the `SHIELD_SETTINGS` dictionary to the value from the form.
        * It **redirects** back to the same page (`/shield_calibration`), so we can verify that the power level value has actually changed.

In [ ]:
# configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' # setting the "secret key"

In [ ]:
from flask import Flask, render_template, redirect, url_for
from config.configuration import Config
from forms import ShieldForm


SHIELD_SETTINGS = {'power_level': 50, 'frequency': 75.5}

app = Flask(__name__)

app.config.from_object(Config)


@app.route("/shield_calibration", methods=["GET", "POST"])
def shield_calibration():
    form = ShieldForm()
    if form.validate_on_submit():
        SHIELD_SETTINGS['power_level'] = form.new_power_level.data
        return redirect(url_for('shield_calibration'))
    return render_template('shield_calibration.html', form=form, power_level=SHIELD_SETTINGS['power_level'])    


if __name__ == '__main__':
    app.run()

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import IntegerField, SubmitField, validators

class ShieldForm(FlaskForm):
    new_power_level = IntegerField('Power level', validators=[validators.DataRequired()])
    submit = SubmitField('Submit')

In [ ]:
<!-- shield_calibration.html -->

<form method = "POST" action = "{{ url_for('shield_calibration') }}">
    <p>Shield calibration form</p>
    {{ form.hidden_tag() }} <!-- CSRF token -->
    <p>Power level: {{ power_level }}</p>
    <p>{{ form.new_power_level.label }} {{ form.new_power_level(size=30) }}</p>
    <p>{{ form.submit() }}</p>
</form>

### **Project (Homework): Crew Member Recruitment**

**Mission:** As the mission demands grow, new specialists need to be brought on board. Our task is to create an **onboarding form** for Mission Control and connect it to our application.

`Basic application structure`:
- my_project/
  - my_app.py
  - routes.py
  - **forms.py**
  - templates/
  - static/
  - config/

1.  **New Form:**
    - Create a new file `forms.py` and in it, create a class, e.g., `NewCrewMemberForm`, which will contain fields for entering a new crew member (i.e., `pilot_name`, `specialization`, etc.).

2.  **Route for the Form:**
    - Create a new route `/onboarding` that will accept `GET` and `POST` requests and will display and process this recruitment form.

3.  **Logic Upgrade:**
    - After a successful form submission (`validate_on_submit`), take the data and **add a new member** to our crew.
    - Arrange for them to automatically get an `id` that follows on from the `id` of the last added member (if the last record has an `id` value of "4", the next record gets "5", and the next "6" ..) and for the `status` to be set to `pending`.

4.  **Navigation Upgrade:**
    - Add the `/onboarding` route to the navigation menu.

5.  **Output Check:**
    - After adding a new member, **redirect** the applicant from the `/onboarding` route to the `/crew_members` page and check the updated crew list.

In [ ]:
# configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True 
    SECRET_KEY = 'my_secret_something' 

In [ ]:
from flask import Flask
from config.configuration import Config 
from routes import register_routes

my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}


app = Flask(__name__)

# -------------- CONFIGURATION --------------
# ze souboru pomocí "třídy"
app.config.from_object(Config)

# -------------- INITIALIZATION --------------
register_routes(app, my_crew, my_asteroid)


# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

In [ ]:
# routes.py
from flask import render_template, redirect, url_for
from forms import NewCrewMemberForm # import our form class for "onboarding"


def register_routes(app, my_crew, my_asteroid):
    @app.route("/")
    def homepage():
        return render_template("homepage.html")

    @app.route("/mission_briefing")
    def mission_briefing():
        return render_template("mission_briefing.html")

    @app.route("/target_asteroid")
    def target_asteroid():
        return render_template("target_asteroid.html", my_asteroid=my_asteroid)

    @app.route("/crew_members")
    def crew_members():
        return render_template("crew_members.html", my_crew=my_crew)

    @app.route("/crew_members/<int:id>")
    def crew_members_details(id):
        for crew_member in my_crew:
            if crew_member["id"] == id:
                return render_template("crew_members_details.html", crew_member=crew_member)
        return "Crew member not found"
    
    @app.route("/onboarding", methods=["GET", "POST"])
    def onboarding():
        form = NewCrewMemberForm()
        if form.validate_on_submit():
            newbie_id = my_crew[-1]["id"] + 1
            newbie_status = "Pending.."
            
            my_crew.append({
                "id": newbie_id,
                "name": form.name.data,
                "specialization": form.specialization.data,
                "status": newbie_status
            })
            return redirect(url_for("crew_members"))
        return render_template("onboarding.html", form=form)

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class NewCrewMemberForm(FlaskForm):
    name = StringField(label="Name", validators=[validators.DataRequired()])
    specialization = StringField(label="Specialization", validators=[validators.DataRequired()])
    
    submit = SubmitField(label="Submit new crew member")

## /templates

In [ ]:
<!-- header.html -->
 
<nav>
    <ul>
        <li><a href="{{ url_for('homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('target_asteroid') }}">The Asteroid</a></li>
        <li><a href="{{ url_for('crew_members') }}">Our Crew</a></li>
        <li><a href="{{ url_for('onboarding') }}">New Recruits Onboarding</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->
 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> 
    
</head>

<body>
    {% include 'header.html' %} 
    {% block page_content %} Content {% endblock page_content %} 
    {% include 'footer.html' %} 
</body>

</html>

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} 

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %} 

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %} 

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/>
{% endblock page_content %}

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} 

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %}

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}
        <p><a href="{{ url_for('crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
    {% endfor %}
{% endblock page_content %}

In [ ]:
<!-- onboarding.html -->

{% extends "base.html" %} <!-- "zdědíme" obsah z base.html -->

{% block page_title %} Onboarding {% endblock page_title %}

{% block page_content %}
<form method = "POST" action = "{{ url_for('onboarding') }}">
    <p>New crew member onboarding form</p>
    {{ form.hidden_tag() }} <!-- CSRF token -->

    <p>{{ form.name.label }} {{ form.name(size=30) }}</p>
    <p>{{ form.specialization.label }} {{ form.specialization(size=30) }}</p>

    <p>{{ form.submit() }}</p>
</form>
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom